In [1]:
import pandas as pd
import numpy as np
import optuna

/home/harkhymadhe/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import data
df = pd.read_csv("game_data_levels.csv")

In [3]:
# Peek at the data
df.head()

,AddMoves,BuyMoreMoves,Churn,DoubleMana,EndType,ExtraMoves,Level,LevelType,LevelVersion,MaxLevel,...,Scores,ServerTime,StartMoves,SubLevel,UsedChangeCar,UsedCoins,UsedMoves,UserID,Variant,WatchedVideo
0,0,0,0,0,Lose,0,80,Saga,6,80,...,2300,2019-01-01 00:00:08,22,50,0,0,22,0UAqFzWsDK4FrUMp48Y3,0,0
1,0,0,0,0,Lose,0,275,Saga,8,275,...,2865,2019-01-01 00:00:14,13,50,0,0,13,tT3QDgAL47D1qXIaSyZP,0,0
2,0,1,0,0,Win,5,4226,Saga,0,4226,...,4560,2019-01-01 00:00:16,21,0,0,27,26,aE1pu1lJo7XBetF5gIRH,0,0
3,0,0,0,0,Lose,0,131,Saga,0,131,...,2970,2019-01-01 00:00:27,13,50,0,0,13,YH7LKBKxJsllLODfU250,0,0
4,0,0,0,0,Lose,0,131,Saga,0,131,...,2180,2019-01-01 00:00:29,17,50,0,0,17,35HyRrY03K6JBO94XfLE,0,0


In [4]:
# Label distribution
df['Churn'].value_counts()

Churn
0    784503
1      5376
Name: count, dtype: int64

In [5]:
df.columns

Index(['AddMoves', 'BuyMoreMoves', 'Churn', 'DoubleMana', 'EndType',
       'ExtraMoves', 'Level', 'LevelType', 'LevelVersion', 'MaxLevel',
       'NextCar', 'Platform', 'PlayTime', 'RollingLosses', 'Scores',
       'ServerTime', 'StartMoves', 'SubLevel', 'UsedChangeCar', 'UsedCoins',
       'UsedMoves', 'UserID', 'Variant', 'WatchedVideo'],
      dtype='object')

In [6]:
len(df.columns)

24

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789879 entries, 0 to 789878
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   AddMoves       789879 non-null  int64  
 1   BuyMoreMoves   789879 non-null  int64  
 2   Churn          789879 non-null  int64  
 3   DoubleMana     789879 non-null  int64  
 4   EndType        789879 non-null  object 
 5   ExtraMoves     789879 non-null  int64  
 6   Level          789879 non-null  int64  
 7   LevelType      789879 non-null  object 
 8   LevelVersion   789879 non-null  int64  
 9   MaxLevel       789879 non-null  int64  
 10  NextCar        789879 non-null  int64  
 11  Platform       789879 non-null  object 
 12  PlayTime       789879 non-null  int64  
 13  RollingLosses  789879 non-null  float64
 14  Scores         789879 non-null  int64  
 15  ServerTime     789879 non-null  object 
 16  StartMoves     789879 non-null  int64  
 17  SubLevel       789879 non-nul

In [8]:
# Convert from string to datetime format
df["ServerTime"] = pd.to_datetime(df["ServerTime"])
df["ServerTime"] = pd.to_datetime(df["ServerTime"], format="%Y-%m-%d")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789879 entries, 0 to 789878
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   AddMoves       789879 non-null  int64         
 1   BuyMoreMoves   789879 non-null  int64         
 2   Churn          789879 non-null  int64         
 3   DoubleMana     789879 non-null  int64         
 4   EndType        789879 non-null  object        
 5   ExtraMoves     789879 non-null  int64         
 6   Level          789879 non-null  int64         
 7   LevelType      789879 non-null  object        
 8   LevelVersion   789879 non-null  int64         
 9   MaxLevel       789879 non-null  int64         
 10  NextCar        789879 non-null  int64         
 11  Platform       789879 non-null  object        
 12  PlayTime       789879 non-null  int64         
 13  RollingLosses  789879 non-null  float64       
 14  Scores         789879 non-null  int64         
 15  

In [10]:
df.head()

,AddMoves,BuyMoreMoves,Churn,DoubleMana,EndType,ExtraMoves,Level,LevelType,LevelVersion,MaxLevel,...,Scores,ServerTime,StartMoves,SubLevel,UsedChangeCar,UsedCoins,UsedMoves,UserID,Variant,WatchedVideo
0,0,0,0,0,Lose,0,80,Saga,6,80,...,2300,2019-01-01 00:00:08,22,50,0,0,22,0UAqFzWsDK4FrUMp48Y3,0,0
1,0,0,0,0,Lose,0,275,Saga,8,275,...,2865,2019-01-01 00:00:14,13,50,0,0,13,tT3QDgAL47D1qXIaSyZP,0,0
2,0,1,0,0,Win,5,4226,Saga,0,4226,...,4560,2019-01-01 00:00:16,21,0,0,27,26,aE1pu1lJo7XBetF5gIRH,0,0
3,0,0,0,0,Lose,0,131,Saga,0,131,...,2970,2019-01-01 00:00:27,13,50,0,0,13,YH7LKBKxJsllLODfU250,0,0
4,0,0,0,0,Lose,0,131,Saga,0,131,...,2180,2019-01-01 00:00:29,17,50,0,0,17,35HyRrY03K6JBO94XfLE,0,0


In [11]:
df.nunique()

AddMoves              1
BuyMoreMoves          8
Churn                 2
DoubleMana            1
EndType               4
ExtraMoves            8
Level              6240
LevelType             1
LevelVersion         35
MaxLevel           6230
NextCar               1
Platform              1
PlayTime           2562
RollingLosses       618
Scores             1779
ServerTime       737698
StartMoves           38
SubLevel              4
UsedChangeCar         1
UsedCoins           131
UsedMoves            60
UserID             4770
Variant               4
WatchedVideo          1
dtype: int64

In [12]:
df.describe()

,AddMoves,BuyMoreMoves,Churn,DoubleMana,ExtraMoves,Level,LevelVersion,MaxLevel,NextCar,PlayTime,RollingLosses,Scores,ServerTime,StartMoves,SubLevel,UsedChangeCar,UsedCoins,UsedMoves,Variant,WatchedVideo
count,789879.0,789879.000000,789879.000000,789879.0,789879.000000,789879.000000,789879.000000,789879.000000,789879.0,789879.000000,789879.000000,789879.000000,789879,789879.000000,789879.000000,789879.0,789879.000000,789879.000000,789879.000000,789879.0
mean,0.0,0.045677,0.006806,0.0,0.228383,474.050791,6.794656,474.970051,0.0,127.187226,11.865531,3218.710037,2019-02-24 22:51:47.251590656,18.936312,36.856572,0.0,1.388419,18.024115,0.088360,0.0
min,0.0,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,11.000000,0.0,2.000000,0.000000,0.000000,2019-01-01 00:00:08,5.000000,0.000000,0.0,0.000000,-5.000000,0.000000,0.0
25%,0.0,0.000000,0.000000,0.0,0.000000,66.000000,0.000000,67.000000,0.0,84.000000,1.000000,2445.000000,2019-01-26 16:49:30.500000,15.000000,0.000000,0.0,0.000000,14.000000,0.000000,0.0
50%,0.0,0.000000,0.000000,0.0,0.000000,142.000000,6.000000,142.000000,0.0,107.000000,3.000000,3050.000000,2019-02-24 23:16:40,19.000000,50.000000,0.0,0.000000,18.000000,0.000000,0.0
75%,0.0,0.000000,0.000000,0.0,0.000000,386.000000,10.000000,387.000000,0.0,137.000000,10.000000,3870.000000,2019-03-23 01:28:24.500000,22.000000,50.000000,0.0,0.000000,22.000000,0.000000,0.0
max,0.0,7.000000,1.000000,0.0,35.000000,6240.000000,34.000000,6240.000000,0.0,88938.000000,617.000000,11300.000000,2019-04-30 23:59:53,42.000000,50.000000,0.0,238.000000,59.000000,3.000000,0.0
std,0.0,0.240063,0.082218,0.0,1.200317,931.899420,6.546539,931.875899,0.0,291.939344,29.015963,1134.421662,NaN,5.250793,22.009010,0.0,5.534455,6.094208,0.328264,0.0


In [13]:
def generate_temporal_features(date):
    """Generate temporal features from timestamp string"""

    date = pd.Timestamp(date)

    return pd.Series(
        [
            date.year,
            date.quarter,
            date.is_quarter_start,
            date.is_quarter_end,
            date.month,
            date.is_month_start,
            date.is_month_end,
            date.week,
            date.day,
            date.day_of_year,
            date.day_of_week,
            date.hour,
            date.minute,
        ],
        index=[
            "year",
            "quarter",
            "quarter_start",
            "quarter_end",
            "month",
            "month_start",
            "month_end",
            "week",
            "day",
            "day_in_year",
            "day_of_week",
            "hour",
            "minute",
        ],
    )

In [14]:
# Converted to datetime
df["ServerTime"] = pd.to_datetime(df["ServerTime"])

In [15]:
new_df = df["ServerTime"].apply(generate_temporal_features)

In [16]:
final_df = pd.concat([df, new_df], axis=1)

In [17]:
final_df.isnull().sum()

AddMoves         0
BuyMoreMoves     0
Churn            0
DoubleMana       0
EndType          0
ExtraMoves       0
Level            0
LevelType        0
LevelVersion     0
MaxLevel         0
NextCar          0
Platform         0
PlayTime         0
RollingLosses    0
Scores           0
ServerTime       0
StartMoves       0
SubLevel         0
UsedChangeCar    0
UsedCoins        0
UsedMoves        0
UserID           0
Variant          0
WatchedVideo     0
year             0
quarter          0
quarter_start    0
quarter_end      0
month            0
month_start      0
month_end        0
week             0
day              0
day_in_year      0
day_of_week      0
hour             0
minute           0
dtype: int64

In [18]:
final_df.nunique()

AddMoves              1
BuyMoreMoves          8
Churn                 2
DoubleMana            1
EndType               4
ExtraMoves            8
Level              6240
LevelType             1
LevelVersion         35
MaxLevel           6230
NextCar               1
Platform              1
PlayTime           2562
RollingLosses       618
Scores             1779
ServerTime       737698
StartMoves           38
SubLevel              4
UsedChangeCar         1
UsedCoins           131
UsedMoves            60
UserID             4770
Variant               4
WatchedVideo          1
year                  1
quarter               2
quarter_start         2
quarter_end           1
month                 4
month_start           2
month_end             2
week                 18
day                  30
day_in_year         118
day_of_week           7
hour                 24
minute               60
dtype: int64

In [19]:
## Remove irrelevant columns
redundant_labels = final_df.nunique()[final_df.nunique() < 2].index
final_df.drop(labels = redundant_labels, axis=1, inplace=True)

In [20]:
y = final_df['Churn']
X = final_df.drop(labels=["UserID", "Churn", "ServerTime"], axis=1)

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789879 entries, 0 to 789878
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   BuyMoreMoves   789879 non-null  int64  
 1   EndType        789879 non-null  object 
 2   ExtraMoves     789879 non-null  int64  
 3   Level          789879 non-null  int64  
 4   LevelVersion   789879 non-null  int64  
 5   MaxLevel       789879 non-null  int64  
 6   PlayTime       789879 non-null  int64  
 7   RollingLosses  789879 non-null  float64
 8   Scores         789879 non-null  int64  
 9   StartMoves     789879 non-null  int64  
 10  SubLevel       789879 non-null  int64  
 11  UsedCoins      789879 non-null  int64  
 12  UsedMoves      789879 non-null  int64  
 13  Variant        789879 non-null  int64  
 14  quarter        789879 non-null  int64  
 15  quarter_start  789879 non-null  bool   
 16  month          789879 non-null  int64  
 17  month_start    789879 non-nul

In [22]:
X.nunique()

BuyMoreMoves        8
EndType             4
ExtraMoves          8
Level            6240
LevelVersion       35
MaxLevel         6230
PlayTime         2562
RollingLosses     618
Scores           1779
StartMoves         38
SubLevel            4
UsedCoins         131
UsedMoves          60
Variant             4
quarter             2
quarter_start       2
month               4
month_start         2
month_end           2
week               18
day                30
day_in_year       118
day_of_week         7
hour               24
minute             60
dtype: int64

In [23]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline

In [24]:
from sklearn import preprocessing

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y)

In [27]:
y_train.shape

(552915,)

In [28]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [29]:
def objective(trial):
    # Combine One hot encoding and stadard scaler to form one higher level transformer
    first_transformer = ColumnTransformer(
        transformers = [
            # Transformer name, transformer object, column index the transformer will work on
            ("endTypeEncoder", OneHotEncoder(handle_unknown="infrequent_if_exist"), (1, 4, 13, 14, 15, 17, 18)),
            ("levelEncoder", StandardScaler(), (3, 5, 6, 8)),
        ],
        remainder="passthrough"
    )
    
    second_transformer = ColumnTransformer(
        transformers = [
            # Transformer name, transformer object, column index the transformer will work on
            ("endTypeEncoder", OneHotEncoder(handle_unknown="infrequent_if_exist"), (1, 4, 13, 14, 15, 17, 18)),
            ("levelEncoder", RobustScaler(), (3, 5, 6, 8)),
        ],
        remainder="passthrough"
    )
    
    log_params = {
        "max_iter": 20000,
        "solver": "saga",
        "class_weight": trial.suggest_categorical("class_weight", choices=[None, "balanced"]),
        "C": trial.suggest_float("C", .1, 2.0),
        "penalty": trial.suggest_categorical("penalty", choices=["l1", "l2", "elasticnet"]),
        "l1_ratio": trial.suggest_float("l1_ratio", 0., 1.0)
    }
    smote_params = {
        "sampling_strategy": trial.suggest_float("sampling_strategy", .1, 1.0)
    }
    pca_params = {
        "n_components": trial.suggest_categorical("n_components", choices=range(10, 26)),
        "whiten": trial.suggest_categorical("whiten", choices=[True, False]),
        "iterated_power": trial.suggest_categorical("iterated_power", choices = range(1, 5)),
        "svd_solver": trial.suggest_categorical("svd_solver", choices=['full', 'arpack', 'randomized'])
    }

    model = Pipeline(
        steps = [
            ("transform", FeatureUnion(transformer_list = [
                ("first", first_transformer),
                ("second", second_transformer)
            ])), # Transform data
            ("reduce", PCA(**pca_params)),
            ("tomek", TomekLinks()),
            ("oversample", SMOTETomek(**smote_params)), # Oversample; solve data imbalance
            ("model", LogisticRegression(**log_params))
        ]
    )
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    metric = roc_auc_score(y_test, predictions)
    return metric

In [30]:
study_name = "log_reg"
storage_url = "sqlite:///log_reg_study.db"

study = optuna.create_study(direction='maximize', study_name=study_name, storage=storage_url, load_if_exists=True)

[I 2023-08-22 12:14:03,751] Using an existing study with name 'log_reg' instead of creating a new one.


In [ ]:
n_trials = 100
study.optimize(objective, n_trials=n_trials)

/home/harkhymadhe/mambaforge/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
[I 2023-08-22 12:32:55,048] Trial 28 finished with value: 0.6585546759941832 and parameters: {'class_weight': 'balanced', 'C': 0.28173416358813663, 'penalty': 'l1', 'l1_ratio': 0.7933610198970351, 'sampling_strategy': 0.7070531815057719, 'n_components': 17, 'whiten': True, 'iterated_power': 3, 'svd_solver': 'full'}. Best is trial 28 with value: 0.6585546759941832.
/home/harkhymadhe/mambaforge/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1171: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
[I 2023-08-22 12:48:31,386] Trial 29 finished with value: 0.6287599171598344 and parameters: {'class_weight': None, 'C': 0.2703354101420932, 'penalty': 'l1', 'l1_ratio': 0.6483844068658199, 'sampling_strategy': 0.712654

In [ ]:
def build_model(best_params):
    # Combine One hot encoding and stadard scaler to form one higher level transformer
    smote_params = {
        k: v for (k, v) in best_params.items()
        if k in ["sampling_strategy"]
    }
    log_params = {
        k: v for (k, v) in best_params.items()
        if k in ["max_iter", "C", "class_weight", "penalty", "l1_ratio", "solver"]
    }
    pca_params = {
        k: v for (k, v) in best_params.items()
        if k not in ["max_iter", "C", "class_weight", "penalty", "l1_ratio", "solver", "sampling_strategy"]
    }
    
    first_transformer = ColumnTransformer(
        transformers = [
            # Transformer name, transformer object, column index the transformer will work on
            ("endTypeEncoder", OneHotEncoder(), (1, 4, 13, 14, 15, 17, 18)),
            ("levelEncoder", StandardScaler(), (3, 5, 6, 8)),
        ],
        remainder="passthrough"
    )
    
    second_transformer = ColumnTransformer(
        transformers = [
            # Transformer name, transformer object, column index the transformer will work on
            ("endTypeEncoder", OneHotEncoder(), (1, 4, 13, 14, 15, 17, 18)),
            ("levelEncoder", RobustScaler(), (3, 5, 6, 8)),
        ],
        remainder="passthrough"
    )


    model = Pipeline(
        steps = [
            ("transform", FeatureUnion(transformer_list = [
                ("first", first_transformer),
                ("second", second_transformer)
            ])), # Transform data
            ("reduce", PCA(**pca_params)),
            ("tomek", TomekLinks()),
            ("oversample", SMOTETomek(**smote_params)), # Oversample; solve data imbalance
            ("model", LogisticRegression(max_iter = 20_000, solver = "saga", **log_params))
        ]
    )
    
    return model

In [ ]:
try:
    best_params = study.best_params
except:
    best_params = {'class_weight': 'balanced',
 'C': 1.9656444719074713,
 'penalty': 'l1',
 'l1_ratio': 0.47294706547245646,
 'sampling_strategy': 0.1042082926173874,
 'n_components': 18,
 'whiten': True,
 'iterated_power': 1,
 'svd_solver': 'full'}

In [ ]:
# Fit on training data
model = build_model(best_params)
model.fit(X_train, y_train)

In [ ]:
model.named_steps["model"].coef_.shape

Do not tresspass

---

In [ ]:
### Train ROC AUC
roc_auc_score(y_train, model.predict(X_train),)

In [ ]:
### Test ROC AUC
roc_auc_score(y_test, model.predict(X_test),)

---

In [ ]:
### Train ROC AUC
roc_auc_score(y_train, model.predict(X_train),)

In [ ]:
### Test ROC AUC
roc_auc_score(y_test, model.predict(X_test),)

---

In [ ]:
### Train accuracy
accuracy_score(y_train, model.predict(X_train),)

In [ ]:
### Test accuracy
accuracy_score(y_test, model.predict(X_test),)

In [ ]:
confusion_matrix(y_test, model.predict(X_test))

In [ ]:
print(classification_report(y_test, model.predict(X_test)))

### Report Outline

1. Discuss the steps for any data project.
    - Business Understanding
    - Data Understanding
    - Data Preparation
    - Data Modelling
    - Model validation
    - Deployment
<br>
<br>
2. Data Imbalance (Data + Learning Algorithm + Evaluation Metric =  ML Product)
    - Data
        - Data duplication or deduplication
        - Oversampling (SMOTE, ADASYN etcetera)
        - Undersampling (UnderMiss, NearMiss etc)
        - Over-Undersampling
    - Learning Algorithm (Cost-Sensitive Learning Algorithms)
        - Log Reg
        - SVMs
        - Decison Trees (Random Forest etcetera)
        - Make use `class_weight` parameter
    - Evaluation Metric
        - Precision
        - AUC score
        - Recall
        - PR AUC curve

Has Cancer but Doctor says they dont -> Cost high, False Negative
Has does not have Cancer but Doctor says they do -> Cost not as high, False Positive

In [ ]:
ohe = OneHotEncoder()

In [ ]:
colors = pd.DataFrame(["red", "green", "blue", "red"])

In [ ]:
colors

In [ ]:
ohe.fit(colors)

In [ ]:
ohe.transform(colors).todense()

In [ ]:
colors

In [ ]:
ohe.categories_

In [ ]:
ohe.

### Next Steps

1. Explore the possibility of generating more features from the `ServerTime` column.
2. Explore more avenues for treating the data imbalance.
3. Do some more EDA to better understand the data features and how they:
    - relate to each other (e.g. correlation)
    - how important they are for final prediction (e.g. feature importance)
    - Remove columns with a cardinality of 1:
        - `LevelType` column. It only contains one value (Saga).
        - `AddMoves` column. It only contains one value (0).
        - `DoubleMana` column. It only contains one value (0).
        - `NextCar` column. It only contains one value (0).

In [ ]:
X_train.nunique()